# Tried wih DistilBERT Model 

In [ ]:
# Install necessary libraries
!pip install torch pandas scikit-learn transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, AdamW
import re
import string
from tqdm.auto import tqdm

In [ ]:
# Check if GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load the dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# Display basic information about the datasets
print("Training Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())

Training Data:
   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  

Test Data:
   id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and T

In [ ]:
# Function to display missing values
def display_missing_values(data, dataset_name):
    print(f"\nMissing Values in {dataset_name}:")
    missing = data.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print(missing)
    else:
        print("No missing values.")

In [ ]:
# Display missing values before preprocessing
display_missing_values(train_data, "Training Data")
display_missing_values(test_data, "Test Data")


Missing Values in Training Data:
keyword       61
location    2533
dtype: int64

Missing Values in Test Data:
keyword       26
location    1105
dtype: int64


In [ ]:
# Preprocessing function
def preprocess_data(data, dataset_name):
    # Handle 'text' column
    if 'text' in data.columns:
        # Fill missing text with empty strings
        data['text'] = data['text'].fillna('').astype(str)

        # Convert text to lowercase
        data['text'] = data['text'].str.lower()

        # Remove URLs
        data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+|www\.\S+', '', x))

        # Remove punctuation
        data['text'] = data['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

        # Remove numbers
        data['text'] = data['text'].apply(lambda x: re.sub(r'\d+', '', x))

        # Remove extra whitespace
        data['text'] = data['text'].apply(lambda x: ' '.join(x.split()))

    # Handle other columns if any
    for column in data.columns:
        if column not in ['id', 'text', 'target']:
            if data[column].dtype == 'object':
                # For categorical columns, fill missing values with the mode
                mode = data[column].mode()[0]
                data[column] = data[column].fillna(mode)
            else:
                # For numerical columns, fill missing values with the median
                median = data[column].median()
                data[column] = data[column].fillna(median)

    # After handling, check if any missing values remain
    missing_after = data.isnull().sum()
    missing_after = missing_after[missing_after > 0]
    if not missing_after.empty:
        print(f"\nWarning: There are still missing values in {dataset_name}:")
        print(missing_after)
    else:
        print(f"\nAll missing values handled in {dataset_name}.")

    return data

In [ ]:
# Apply preprocessing
train_data = preprocess_data(train_data, "Training Data")
test_data = preprocess_data(test_data, "Test Data")


All missing values handled in Training Data.

All missing values handled in Test Data.


In [ ]:
# Display missing values after preprocessing
display_missing_values(train_data, "Training Data")
display_missing_values(test_data, "Test Data")


Missing Values in Training Data:
No missing values.

Missing Values in Test Data:
No missing values.


In [ ]:
# Ensure there are no missing values in 'target' for training data
if train_data['target'].isnull().sum() > 0:
    print("\nMissing values found in 'target' column. Removing corresponding rows.")
    train_data = train_data.dropna(subset=['target'])
    print(f"New training data shape: {train_data.shape}")
else:
    print("\nNo missing values in 'target' column.")


No missing values in 'target' column.


In [ ]:
# Reset index after dropping rows
train_data = train_data.reset_index(drop=True)

In [ ]:
# Check for duplicates and remove if necessary
def remove_duplicates(data, dataset_name):
    initial_shape = data.shape
    data = data.drop_duplicates()
    final_shape = data.shape
    print(f"\nDuplicates in {dataset_name}: {initial_shape[0] - final_shape[0]}")
    return data

train_data = remove_duplicates(train_data, "Training Data")
test_data = remove_duplicates(test_data, "Test Data")


Duplicates in Training Data: 0

Duplicates in Test Data: 0


In [ ]:
# Display sample after preprocessing
print("\nSample Preprocessed Training Data:")
print(train_data.head())


Sample Preprocessed Training Data:
   id     keyword location                                               text  \
0   1  fatalities      USA  our deeds are the reason of this earthquake ma...   
1   4  fatalities      USA              forest fire near la ronge sask canada   
2   5  fatalities      USA  all residents asked to shelter in place are be...   
3   6  fatalities      USA  people receive wildfires evacuation orders in ...   
4   7  fatalities      USA  just got sent this photo from ruby alaska as s...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
# Check class distribution
class_counts = train_data['target'].value_counts()
print("\nClass Distribution in Training Data:")
print(class_counts)


Class Distribution in Training Data:
target
0    4342
1    3271
Name: count, dtype: int64


In [ ]:
# Calculate class weights to handle imbalance
classes = np.unique(train_data['target'])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_data['target'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("\nClass Weights:", class_weights)


Class Weights: tensor([0.8767, 1.1637], device='cuda:0')


In [ ]:
# Split the training data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['text'].tolist(),
    train_data['target'].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=train_data['target']
)

print(f"\nNumber of training samples: {len(train_texts)}")
print(f"Number of validation samples: {len(val_texts)}")


Number of training samples: 6851
Number of validation samples: 762


In [ ]:
# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Tokenization parameters
MAX_LEN = 256

# Tokenize training data
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

In [ ]:
# Tokenize validation data
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

In [ ]:
# Tokenize test data
test_encodings = tokenizer(
    test_data['text'].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LEN
)

In [ ]:
# Define a custom Dataset class
class DisasterDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# Create Dataset objects
train_dataset = DisasterDataset(encodings=train_encodings, labels=train_labels)
val_dataset = DisasterDataset(encodings=val_encodings, labels=val_labels)
test_dataset = DisasterDataset(encodings=test_encodings)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mod

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc="Training", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    return avg_loss

In [ ]:
# Evaluation function
def evaluate(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    preds = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = loss_fn(logits, labels)
            total_loss += loss.item()

            predictions = torch.argmax(logits, dim=1)
            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    f1 = f1_score(true_labels, preds)
    report = classification_report(true_labels, preds, digits=4)
    return avg_loss, f1, report

In [ ]:
# Early Stopping parameters
patience = 2
best_f1 = 0.0
epochs_no_improve = 0
EPOCHS = 100

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-" * 30)

    train_loss = train(model, train_loader, optimizer, loss_fn)
    print(f"Training Loss: {train_loss:.4f}")

    val_loss, val_f1, val_report = evaluate(model, val_loader, loss_fn)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation F1 Score: {val_f1:.4f}")
    print("Classification Report:")
    print(val_report)

    # Save the model if validation F1 improves
    if val_f1 > best_f1:
        best_f1 = val_f1
        model.save_pretrained('best_distilbert_model3')
        tokenizer.save_pretrained('best_distilbert_model3')
        print("Best model saved.")


Epoch 1/100
------------------------------


Training:   0%|          | 0/108 [00:00<?, ?it/s]

Training Loss: 0.0253


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

Validation Loss: 1.0474
Validation F1 Score: 0.8019
Classification Report:
              precision    recall  f1-score   support

           0     0.8465    0.8621    0.8542       435
           1     0.8119    0.7920    0.8019       327

    accuracy                         0.8320       762
   macro avg     0.8292    0.8271    0.8280       762
weighted avg     0.8317    0.8320    0.8317       762

Best model saved.

Epoch 2/100
------------------------------


...

Validation Loss: 1.0474
Validation F1 Score: 0.8019
Classification Report:
              precision    recall  f1-score   support

           0     0.8465    0.8621    0.8542       435
           1     0.8119    0.7920    0.8019       327

    accuracy                         0.8320       762
   macro avg     0.8292    0.8271    0.8280       762
weighted avg     0.8317    0.8320    0.8317       762


Epoch 100/100
------------------------------


Training:   0%|          | 0/108 [00:00<?, ?it/s]

Training Loss: 0.0256


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

Validation Loss: 1.0474
Validation F1 Score: 0.8019
Classification Report:
              precision    recall  f1-score   support

           0     0.8465    0.8621    0.8542       435
           1     0.8119    0.7920    0.8019       327

    accuracy                         0.8320       762
   macro avg     0.8292    0.8271    0.8280       762
weighted avg     0.8317    0.8320    0.8317       762



In [ ]:
# Load the best model
model = DistilBertForSequenceClassification.from_pretrained('best_distilbert_model3')
model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
# Predict on the test set
model.eval()
test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting", leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        test_preds.extend(predictions.cpu().numpy())

# Prepare the submission
output = pd.DataFrame({
    'id': test_data['id'],
    'target': test_preds
})

Predicting:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
# Save the predictions to a CSV file
output.to_csv('distiloutput3.csv', index=False)
print("\nPredictions saved to distiloutput3.csv")


Predictions saved to distiloutput3.csv
